In [183]:
# Import Dependencies
import pandas as pd
import numpy as np

In [184]:
# Create a path to the JSON and read it into a Pandas DataFrame
json_path = "purchase_data.json"
purchase_data_df = pd.read_json(json_path, orient = "records")

purchase_data_df.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [185]:
#Total Players
player_demos = purchase_data_df.loc[:,["SN","Gender","Age"]]

#Drop Duplicates
player_demos = player_demos.drop_duplicates()
total_players= player_demos.count()[0]
total_players_df = pd.DataFrame({"Total Players":[total_players]})
total_players_df

,Total Players
0,573


In [186]:
#Purchasing Analysis (Total)

item_count = len(purchase_data_df["Item ID"].unique())

#Calculate the Average Purchase Price
avg_price = purchase_data_df["Price"].mean()
#Calculate Total Number of Purchases
tot_purchases = purchase_data_df["Item ID"].count()
tot_revenue = purchase_data_df["Price"].sum()
purchasing_analysis = pd.DataFrame({"Total Unique Items":[item_count],
                                    "Avg Purchase Price":[avg_price],
                                    "Total Purchases":[tot_purchases],
                                    "Total Revenue":[tot_revenue]})
purchasing_analysis["Avg Purchase Price"] = purchasing_analysis["Avg Purchase Price"].map("${:.2f}".format)
purchasing_analysis["Total Revenue"] = purchasing_analysis["Total Revenue"].map("${:.2f}".format)
ord_purchasing_analysis = purchasing_analysis[["Total Unique Items","Avg Purchase Price","Total Purchases","Total Revenue"]]
ord_purchasing_analysis

,Total Unique Items,Avg Purchase Price,Total Purchases,Total Revenue
0,183,$2.93,780,$2286.33


In [187]:
#Gender Demographics
gender_counts = player_demos["Gender"].value_counts().reset_index()
gender_counts.columns = ["Gender","Total Count"]
gender_counts
percent = (gender_counts["Total Count"]/total_players)* 100
gender_counts["Percent of Players"]= percent
ord_gender_counts = gender_counts[["Gender","Percent of Players","Total Count"]]
ord_gender_counts

,Gender,Percent of Players,Total Count
0,Male,81.151832,465
1,Female,17.452007,100
2,Other / Non-Disclosed,1.396161,8


In [188]:
#Purchasing Analysis (Gender)

grouped_purchase_data = purchase_data_df.groupby("Gender",as_index=False).agg({"SN":"count","Price":["mean","sum"]})
grouped_purchase_data.columns = ["Gender","Purchase Count","Average Purchase Price","Total Purchase Value"]
#normalized_total = grouped_purchase_data["Total Purchase Value"]/grouped_purchase_data["Count"]
normalized_total = grouped_purchase_data["Total Purchase Value"]/total_players
grouped_purchase_data["Normalized Totals"]= normalized_total
grouped_purchase_data

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
0,Female,136,2.815515,382.91,0.668255
1,Male,633,2.950521,1867.68,3.259476
2,Other / Non-Disclosed,11,3.249091,35.74,0.062373


In [193]:
#Age Demographics

age_group_purchase_data = purchase_data_df.groupby("Age",as_index=False).agg({"SN":"count","Price":["mean","sum"]})
age_group_purchase_data.columns = ["Age","Purchase Count","Average Purchase Price","Total Purchase Value"]
normalized_total = age_group_purchase_data["Total Purchase Value"]/age_group_purchase_data["Purchase Count"]
#normalized_total = grouped_purchase_data["Total Purchase Value"]/total_players
age_group_purchase_data["Normalized Totals"]= normalized_total

bins = [0,10,14,19,24,29,34,39,44,45]
group_names = ["< 10","10-14","15-19","20-24","25-29","30-34","35-39","40-44","45 >"]
pd.cut(age_group_purchase_data["Age"],bins, labels = group_names)

age_group_purchase_data["Age Summary"]= pd.cut(age_group_purchase_data["Age"],bins,labels = group_names)
age_grouping = age_group_purchase_data.groupby("Age Summary")
age_grouping.max()

,Age,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
Age Summary,,,,,
< 10,10,19,3.686667,55.47,3.686667
10-14,14,11,3.842000,26.76,3.842000
15-19,19,47,3.050000,140.36,3.050000
20-24,24,98,3.049429,282.68,3.049429
25-29,29,67,3.082388,206.52,3.082388
30-34,34,18,3.409091,56.01,3.409091
35-39,39,12,3.085000,37.02,3.085000
40-44,43,14,3.810000,45.11,3.810000
45 >,45,1,2.720000,2.72,2.720000


In [194]:
#Top Spenders
player_spend = purchase_data_df.loc[:,["SN","Item ID","Price"]]
purchase_count = player_spend.groupby(["SN","Item ID"])
purchase_count_df = purchase_count.max()
purchase_price_sum = purchase_count_df.groupby(by=["SN"])["Price"].sum().reset_index()
purchase_price_sum.columns = ["SN","Total Purchase Value"]
purchase_price_mean = purchase_count_df.groupby(by=["SN"])["Price"].mean().reset_index()
purchase_price_mean.columns = ["SN","Average Purchase Price"]
purchase_price_count = purchase_count_df.groupby(by=["SN"])["Price"].count().reset_index() 
purchase_price_count.columns = ["SN","Purchase Count"]
player_spend_df = pd.DataFrame({"SN":purchase_price_sum["SN"],
                                "Purchase Count":purchase_price_count["Purchase Count"],
                                "Average Purchase Price":purchase_price_mean["Average Purchase Price"],
                                "Total Purchase Value":purchase_price_sum["Total Purchase Value"]})
ord_player_spend_df = player_spend_df[["SN","Purchase Count","Average Purchase Price","Total Purchase Value"]]
ord_player_spend_df_sorted = ord_player_spend_df.sort_values("Total Purchase Value", ascending=False)
top_five_spenders_df = ord_player_spend_df_sorted.head()
top_five_spenders_df

,SN,Purchase Count,Average Purchase Price,Total Purchase Value
538,Undirrala66,5,3.412000,17.06
428,Saedue76,4,3.390000,13.56
354,Mindimnya67,4,3.185000,12.74
181,Haellysu29,3,4.243333,12.73
120,Eoda93,3,3.860000,11.58


In [195]:
#Most Popular Items
items_demos = purchase_data_df.loc[:,["Item ID","Item Name"]]
items_count = items_demos["Item ID"].value_counts().reset_index()
#items_df = items_demos["Item Id"].value_counts()
#items_counts = items_df.groupby(["Item ID"])
items_count.columns = ["Item ID","Purchase Count"]
items_count.reset_index()
items_descr = purchase_data_df.loc[:,["Item ID","Item Name","Price"]]
items_df = pd.merge(items_descr, items_count, on="Item ID",how="inner")
items_df = items_df.drop_duplicates()
items_df["Total Purchase Value"] = items_df["Price"] * items_df["Purchase Count"]
items_df
items_df_sorted_items_df = items_df.sort_values("Purchase Count", ascending=False)
most_popular_df = items_df_sorted_items_df.head()
most_popular_df

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
260,39,"Betrayal, Whisper of Grieving Widows",2.35,11,25.85
426,84,Arcane Gem,2.23,11,24.53
331,175,Woeful Adamantite Claymore,1.24,9,11.16
153,13,Serenity,1.49,9,13.41
237,31,Trickster,2.07,9,18.63


In [196]:
#Most Profitable Items
items_df_sorted_items_df = items_df.sort_values("Total Purchase Value", ascending=False)
most_profitable_df = items_df_sorted_items_df.head()
most_profitable_df

,Item ID,Item Name,Price,Purchase Count,Total Purchase Value
246,34,Retribution Axe,4.14,9,37.26
411,115,Spectral Diamond Doomblade,4.25,7,29.75
217,32,Orenmir,4.95,6,29.70
390,103,Singed Scalpel,4.87,6,29.22
525,107,"Splitter, Foe Of Subtlety",3.61,8,28.88
